# Manual Changes

## template mapping files are in the git repository
## original data in _CyVerse Discovery Environment_ 
### data file is: "1987-2019 Cougar Weight-Length Public Request.xlsx"

### _verbatimLocality_
- concatenation of "Management Unit" and "County"

## To Code
### _ageUnit_
- all in years

### _measurementUnit_
- Weight ({body mass}) is in pounds (lbs.) 
- Length ({body length}) is in inches (in.)

### _materialSampleType_
- in "Status" colum
- "A" = Intact 
    - change "A" to "whole"
- "B" = Field Dressed
    - change "B" to "part - gutted"
- "C" = Skinned
    - change "C" to part - skinned